In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from itertools import product

In [2]:
# Demand
df_demand = pd.read_excel('10node.xlsx', 'Customer')
df_demand = df_demand['Demand'].tolist()
df_demand

[60, 27, 29, 26, 33, 15, 17, 97, 97, 19]

In [3]:
# Parameters
p = 4 # covers number
c = 2.5 # covers distance

In [4]:
# Transportation cost
df_dis = pd.read_excel('10node.xlsx', 'Distance')
df_dis = df_dis.iloc[:,1:]
df_dis

,1,2,3,4,5,6,7,8,9,10
0,0.000000,9.219544,3.000000,7.071068,8.544004,4.472136,6.708204,3.605551,5.099020,5.000000
1,9.219544,0.000000,7.615773,5.000000,4.472136,5.000000,3.162278,5.656854,6.082763,7.071068
2,3.000000,7.615773,0.000000,7.280110,5.830952,4.123106,6.000000,3.162278,2.236068,2.000000
3,7.071068,5.000000,7.280110,0.000000,8.062258,3.162278,2.236068,4.123106,7.211103,8.062258
4,8.544004,4.472136,5.830952,8.062258,0.000000,6.082763,5.830952,6.000000,3.605551,4.242641
5,4.472136,5.000000,4.123106,3.162278,6.082763,0.000000,2.236068,1.000000,4.242641,5.000000
6,6.708204,3.162278,6.000000,2.236068,5.830952,2.236068,0.000000,3.162278,5.385165,6.324555
7,3.605551,5.656854,3.162278,4.123106,6.000000,1.000000,3.162278,0.000000,3.605551,4.242641
8,5.099020,6.082763,2.236068,7.211103,3.605551,4.242641,5.385165,3.605551,0.000000,1.000000
9,5.000000,7.071068,2.000000,8.062258,4.242641,5.000000,6.324555,4.242641,1.000000,0.000000


In [5]:
# Number of viable pairings
aij = {(customer, facility): 1 if df_dis.iloc[customer, facility] <= c else 0
            for customer in range(0, 10)
            for facility in range(0, 10)}

print("Number of viable pairings: {0}".format(len(aij.keys())))


Number of viable pairings: 100


In [6]:
m = gp.Model("MCLP")

Restricted license - for non-production use only - expires 2023-10-25


In [7]:
# Decision variables: facilities open or close
fact = m.addVars(10, vtype = GRB.BINARY, name='fact')
z = m.addVars(10, vtype = GRB.BINARY, name='demand')

In [8]:
# Objection
obj = gp.quicksum(z[customer] * df_demand[customer] for customer in range(0, 10))
m.setObjective(obj, GRB.MAXIMIZE)

In [9]:
# Constraints
m.addConstrs((gp.quicksum(aij[(customer, facility)] * fact[facility] for facility in range(0, 10)) >= z[customer] 
              for customer in range(0, 10)), name='coverage distance')
m.addConstr((gp.quicksum(fact[facility] for facility in range(0, 10)) == p) , name='coverage number')



<gurobi.Constr *Awaiting Model Update*>

In [10]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11 rows, 20 columns and 42 nonzeros
Model fingerprint: 0xe081644c
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 221.0000000
Presolve removed 5 rows and 7 columns
Presolve time: 0.00s
Presolved: 6 rows, 13 columns, 24 nonzeros
Variable types: 0 continuous, 13 integer (12 binary)
Found heuristic solution: objective 281.0000000

Root relaxation: objective 3.670000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     367.0000000  367.00000  0.00%     -

In [11]:
# Display optimal values of decision variables

for facility in fact.keys():
    if (abs(fact[facility].x) > 1e-6):
        print(f"\n Build a warehouse at location {facility + 1}.")


 Build a warehouse at location 1.

 Build a warehouse at location 3.

 Build a warehouse at location 5.

 Build a warehouse at location 6.
